In [1]:
import pandas as pd
import PyPDF2
from scripts.functions import getRanges, isDigit

In [2]:
file = './downloads/report-20211201.pdf'
pdfFileObj = open(file,'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

In [3]:
pages = getRanges(file)
incidenza = pages['incidenza']
print(incidenza)

[18, 29]


In [4]:
textes = []

print(incidenza[0], incidenza[1]+1)

try:
    for i in range(incidenza[0], incidenza[1]+1):
            pageObj = pdfReader.getPage(i)
            text = pageObj.extractText()
            text = text.replace('\n', ' ')
            textes.append(text[2::])
    pdfFileObj.close()
except Exception as e:
    pass

18 30


In [5]:
out = ' '.join(textes)\
         .rpartition('settimane')[2]\
         .rpartition('Totale')[0]\
         .replace('- ', '-')\
         .replace('---', '0%')\
         .replace('  ', ' ')\
         .replace('  ', ' ')\
         .replace('  ', ' ')\
         .replace('%', '')\
         .replace("O'","Ò").replace("I'","Ì").replace("U'","Ù")\
         .split()

In [6]:
new = ""

for split in out:
    if not isDigit(split):
        new = new + split + ' '
    if isDigit(split):
        new = new + ','+ split + ','

new = new.replace(',,', ',').replace(' ,', ',').replace(' -','-').split(',')

In [7]:
it = iter(new)
data = list(zip(it, it, it, it))

In [8]:
df = pd.DataFrame(data, columns = ['comune', 'casi', 'incidenza', 'variazione'])
df = df[['comune', 'incidenza', 'casi']]

In [9]:
incidenza = df[~df["comune"].duplicated(keep="last")]
incidenza.reset_index(inplace=True, drop=True)

In [10]:
comuni = pd.DataFrame(pd.read_csv('https://raw.githubusercontent.com/gabacode/vaxExtract/main/utilities/Elenco-comuni-siciliani.csv', converters={'pro_com_t': '{:0>6}'.format}))

In [11]:
out = pd.merge(incidenza, comuni, left_on=incidenza["comune"].str.lower(), right_on=comuni["comune"].str.lower(), how="inner")
out.rename(columns = {'comune_y':'comune'}, inplace = True)
out = out[['cod_prov','pro_com_t','provincia','comune','incidenza','casi']].sort_values(by=['provincia', 'comune'])

In [12]:
out.reset_index(drop=True, inplace=True)
out

,cod_prov,pro_com_t,provincia,comune,incidenza,casi
0,84,084001,Agrigento,Agrigento,57.10,32
1,84,084002,Agrigento,Alessandria della Rocca,0.00,0
2,84,084003,Agrigento,Aragona,22.19,2
3,84,084004,Agrigento,Bivona,0.00,0
4,84,084005,Agrigento,Burgio,38.88,1
...,...,...,...,...,...,...
385,81,081020,Trapani,San Vito Lo Capo,62.41,3
386,81,081019,Trapani,Santa Ninfa,61.54,3
387,81,081021,Trapani,Trapani,139.47,91
388,81,081022,Trapani,Valderice,233.71,27
